### Workflow:
1) pull combinations of current and previous status from the db -> in the form of a contingency table
2) ensure their inclusion in the 'statusCombinationMatrix_acctFlows.ods' file
3) assign values to the scoring vector (1 - for inclusion; 0 - for exclusion)
4) import the .ods file <- use it to build vectors for each scoring category of the Acct Flows table
5) **multiply the vectors**: the "scoring vector" and the freq table from the contingency table from #1. NOTE: ensure that the "freq" vector is in the same order as the scoring vector (sort by the combinations)

In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime
import sys
import sqlalchemy
#edit to the path of the `container_credentials` module
sys.path.append(('/home/mofongo/Documents/ghfc/membershipReportsCIVI/greeneHill'))
from container_credentials import return_credentials

In [2]:
from sqlalchemy import create_engine, Table, text, MetaData # a CORE approach
from sqlalchemy.sql import select
from sqlalchemy.sql import func

In [3]:
os.getcwd()

'/home/mofongo/Documents/ghfc/membershipReportsCIVI/greeneHill/acctFlows'

## Constructing the list of month-end tuples

In [3]:
#get the dates I want in string format
# can conceivably create a matrix of scores (as long as the combos don't change)
def last_day_of_month(any_day):
    # The day 28 exists in every month. 4 days later, it's always next month
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtracting the number of the current day brings us back one month
    return next_month - datetime.timedelta(days=next_month.day)

#for month in range(1, 13):
#    print(last_day_of_month(datetime.date(2023, month, 7)))

#ea of these is a seq that contains all the last days of each month
twentythree = [last_day_of_month(datetime.date(2023, month, 7)) for month in range(1, 13)]
twentyfour = [last_day_of_month(datetime.date(2024, month, 7)) for month in range(1, 13)] 
twentyfive = [last_day_of_month(datetime.date(2025, month, 7)) for month in range(1, 13)] 

In [4]:
#month_end_df = pd.DataFrame({'twentythree':twentythree,'year':2023})
month_end_df = pd.DataFrame({'twentyfour':twentyfour,'year':2024})

In [5]:
#make a replacement column for null values (expect it will only occur above the January record)
month_end_df['prev_yr']=month_end_df.apply(lambda x: str(x['year']-1)+'-12-31', axis = 1)

In [6]:
month_end_shifted = month_end_df.shift(periods=[0,1],axis = 0)
#below may not be necessary

In [7]:
#BE SURE TO CHANGE THE FIELD NAMES TO REFLECT THE MEASURE YEAR
month_end_shifted['twentyfour_1'] = month_end_shifted['twentyfour_1'].fillna(month_end_shifted['prev_yr_0'])
#month_end_shifted['twentythree_1'] = month_end_shifted['twentythree_1'].fillna(month_end_shifted['prev_yr_0'])
month_end_shifted = month_end_shifted.iloc[:,:].astype(str)

In [8]:
# a collection of tuples, ea of which provides the dates combo for the query
#BE SURE TO CHANGE THE FIELD NAMES TO REFLECT THE MEASURE YEAR
calendar_range_iter = [tuple(i.values()) for i in month_end_shifted[['twentyfour_1','twentyfour_0']].to_dict('records')]
#calendar_range_iter = [tuple(i.values()) for i in month_end_shifted[['twentythree_1','twentythree_0']].to_dict('records')]

### Connect to database to gather status combinations and counts

In [9]:
# DEFINE THE DATABASE CREDENTIALS

cred_dict = return_credentials()

user = cred_dict['user'] 
password = cred_dict['pass'] 
host = cred_dict['host'] 
port = cred_dict['port'] 
database = cred_dict['database']

''' legacy code hard coded the credentials
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'
'''

def get_connection():
	return sqlalchemy.create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)

if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		# working w/engines: https://docs.sqlalchemy.org/en/20/core/engines_connections.html
		engine = get_connection() #engine should be created just once, and can manage several DBAPI connections
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)

Connection to the 172.17.0.2 for user root created successfully.


The below query was initially developed in file "membership_queries_dev.sql"

In [10]:
#this query simply returns two snapshots: period 1 AND period 2 activity types, along with freq; ex. curr_activity_calc = 'general leave' prev_activity_calc = 'initial enrollment'
def fill_query(tup_dates:tuple):
    return "WITH curr AS (select mt_email curr_mt_email, mem_type curr_mem_type, activity_calc curr_activity_calc, activity curr_activity from stack_job2 WHERE date('"+tup_dates[1]+"') between start_dt AND lead_date ORDER BY mt_email), prev AS (select mt_email prev_mt_email, mem_type prev_mem_type, activity_calc prev_activity_calc, activity prev_activity from stack_job2 WHERE date('"+tup_dates[0]+"') between start_dt AND lead_date ORDER BY mt_email), final_tbl AS (SELECT date('"+tup_dates[1]+"') current_month, curr_activity_calc, prev_activity_calc, count(distinct curr_mt_email) unq_email FROM curr LEFT JOIN prev ON curr_mt_email = prev_mt_email GROUP BY 1,2,3) SELECT * FROM final_tbl"

In [23]:
## MARGINALLY DIFFERENT QUERY TO THE ONE USED IN fill_query() - it also queries for the last trial type - this will enable counting/filling the currently vacant 'trial_conversion' field of the Account Flows table
#NOTE: source code for table trial_meta_all located in membership_queries_dev.sql
#NOTE 2: the workflow will need to be augmented to run the CREATE TABLE query for trial_meta_all
'''WITH curr AS (
select mt_email curr_mt_email, mem_type curr_mem_type, activity_calc curr_activity_calc, activity curr_activity, tma.last_trial_type 
from stack_job2 
LEFT JOIN trial_meta_all tma ON mt_email = tma.email 
WHERE date('2024-05-31') between start_dt AND lead_date 
GROUP BY 1,2,3,4,5), 
prev AS (
select mt_email prev_mt_email, mem_type prev_mem_type, activity_calc prev_activity_calc, activity prev_activity from stack_job2 WHERE date('2024-04-30') between start_dt AND lead_date ORDER BY mt_email)
SELECT *
FROM curr 
LEFT JOIN prev ON curr_mt_email = prev_mt_email
ORDER BY curr_mt_email'''

"WITH curr AS (\nselect mt_email curr_mt_email, mem_type curr_mem_type, activity_calc curr_activity_calc, activity curr_activity, tma.last_trial_type \nfrom stack_job2 \nLEFT JOIN trial_meta_all tma ON mt_email = tma.email \nWHERE date('2024-05-31') between start_dt AND lead_date \nGROUP BY 1,2,3,4,5), \nprev AS (\nselect mt_email prev_mt_email, mem_type prev_mem_type, activity_calc prev_activity_calc, activity prev_activity from stack_job2 WHERE date('2024-04-30') between start_dt AND lead_date ORDER BY mt_email)\nSELECT *\nFROM curr \nLEFT JOIN prev ON curr_mt_email = prev_mt_email\nORDER BY curr_mt_email"

In [11]:
# a dictionary of the SQL QUERY STRING (the PREPARED STATEMENT), with key = representative month
#dict: key = latest date of the month tuple; value = sql query text
query_cont = {}
for min_tup in calendar_range_iter:
    query_cont[min_tup[1]]=fill_query(min_tup)

In [12]:
#build a dictionary of the result sets, setting the key to the month
df_dict = {}
with engine.connect() as conn:
    for k,query in query_cont.items():
        df_dict[k] = pd.read_sql(query,conn)

TODO: Build a function that creates the tuple index for the "scores" vector. This function can then be iteratively applied to ea result set/DataFrame returned from the db ("df_dict" object)
The key of each dictionary item contains the month of measure

In [13]:
#can theoretically apply this iteratively to ea DataFrame in the result set dict
#function that takes in a dataframe of a month's scores along with activity_calc fields, which then combines the TWO activity_calc fields into a tuple of the status combination. Extracts the unq.email field (int) and converts that into the scores Series with index corresponding to the combo tuple

#TODO can refactor the below function to return TWO Series: the original "scores_series_vector" and a 2nd that returns the count of trials by "combo" tuple; this Series would then follow the same path and be used to multiply against the categories matrix - counting trial members for any category I wish (priority is 'trial_conversion' field)
def package_scores_vector(df: pd.DataFrame):
    #in order to avoid issues with "None" values in the "member status" combinations, convert None to "None" (string)
    df = df.assign(prev_activity_calc = df['prev_activity_calc'].fillna('None'), curr_activity_calc = df['curr_activity_calc'].fillna('None'))

    #make the 'combo' tuple
    test_df = df.assign(combo = [tuple(i.values()) for i in df[['curr_activity_calc','prev_activity_calc']].to_dict('records')])

    #scores_series_vector = the vector that I ultimately multiply by the 'category' matrix (odf) AFTER I compile the index intersection
    scores_series_vector = pd.Series(data = test_df['unq_email'].values, index = test_df['combo'])

    return scores_series_vector

## Creating the .ods file
'df' DataFrame = resultset from db, and contains the counts/frequencies
Don't create the .ods file explicitly. The original file is valuable because I types out the categories manually. If I want to somehow replicate the .ods file programmatically, I'll need to reverse engineer it after importing the pre-existing file.

### only run this when I want to CREATE an ods from scratch

In [22]:
#a list of tuples - ea of which contain distinct combinations of actions
# I copy this output to create the .ods file
df = df.assign(combo = [tuple(i.values()) for i in df[['curr_activity_calc','prev_activity_calc']].to_dict('records')])

#/github/statusCombinationMatrix_acctFlows.ods

#[tuple(i.values()) for i in df[['curr_activity_calc','prev_activity_calc']].to_dict('records')]

## Importing the ODS file 
This file is a matrix table that controls what goes into each category (how to tally sums)

In [14]:
#function to accomplish all processes in this segment: a) import ODS; b) parse and reunify the status tuple that serves as the combo index; c) fillna() properly
# funct argument is filepath to the statusCombinationMatrix_acctFlows.ods file
def import_treat_cat_df(filepath: str):
    #NOTE: the combo tuple will be messed up, and needs to be re-formated and ultimately recognized as a tuple in python
    cats_df = pd.read_excel(filepath, engine="odf")
    cats_df.columns = [i.replace(" ","_").lower() for i in cats_df.columns]

    #use regex to remove the artifacts from the tuple encoding when the .ods file was created from the df copy (done manually)
    pattern = re.compile("[a-zA-Z0-9_ ]+")

    cats_df = cats_df.assign(curr_activity_calc_norm = cats_df['curr_activity_calc'].apply(lambda x: list(re.findall(pattern, x))[-1].strip()), prev_activity_calc_norm = cats_df['prev_activity_calc'].apply(lambda x: list(re.findall(pattern, x))[-1].strip()))

    #create the final tuple column
    cats_df['tuple_index'] = tuple(zip(cats_df['curr_activity_calc_norm'], cats_df['prev_activity_calc_norm']))

    #select relevant fields
    cats_df = cats_df.loc[:,[i for i in list(cats_df.columns) if 'activity' not in i]]

    #set the index
    cats_df.set_index('tuple_index', inplace = True)

    #converts the matrix to a 1,0
    cats_df = cats_df.fillna(0)

    return cats_df

### Importing the categories matrix
(I may not need to run this, as these commands are carried out in a consolidated function)

In [50]:
#NO NEED TO RUN THIS
#import the ods file
#NOTE: the combo tuple will be messed up, and needs to be re-formated and ultimately recognized as a tuple in python
cats_df = pd.read_excel("statusCombinationMatrix_acctFlows.ods", engine="odf")
cats_df.columns = [i.replace(" ","_").lower() for i in cats_df.columns]
#scores_df.columns
#REGEX: Notice how the values returned from the spreadsheet contain artifacts of the tuple and list objects from python. These need to be removed.
#I may want to explore building a function then looping through the Series as the .str suite of functions doesn't help me well.
pattern = re.compile("[a-zA-Z0-9_ ]+")

cats_df = cats_df.assign(curr_activity_calc_norm = cats_df['curr_activity_calc'].apply(lambda x: list(re.findall(pattern, x))[-1].strip()), prev_activity_calc_norm = cats_df['prev_activity_calc'].apply(lambda x: list(re.findall(pattern, x))[-1].strip()))
#create the final tuple column
cats_df['tuple_index'] = tuple(zip(cats_df['curr_activity_calc_norm'], cats_df['prev_activity_calc_norm']))
#Deliver the category matrix
#converts the vector to a 1,0
cats_df = cats_df.fillna(0)

## Procedure to arrange, then multiply the scores vector & category matrix

In [ ]:
#first, work on the result set df: grabbing the 'unq_email' field which contains the scores by combo, set it as its own Series and define the index as the combo values
#predecessor to test_df  is df

#NO NEED TO RUN THIS; THIS IS AN ATOMIC PROROTYPE TO A MORE ABSTRACT FUNCTION 'package_scores_vector()'
scores_series_vector = pd.Series(data = test_df['unq_email'].values, index = test_df['combo'])

In [ ]:
#NO NEED TO RUN THIS
cats_df2 = cats_df.loc[:,[i for i in list(cats_df.columns) if 'activity' not in i]]

cats_df2.set_index('tuple_index', inplace = True)

## Sequence of functions and master function that carry out the matrix-vector multiplication

In [15]:
#function to ingest the matrix and vector, compile the intersection and return the equal sized objects (ready to multiply) in a tuple
# the cat_matrix is the ods file as processed above, terminating in the fillna(0) operation
def sync_objects(scores_vector:pd.Series, cat_df:pd.DataFrame):
    cats_df2_index= list(cat_df.index)
    scores_index = list(scores_vector.index)

    # quantify the # of columns in the scores/contingency vector and NOT in the categorical matrix
    #outer = list(set() - set(cats_df2_index))
    outer = list(set(scores_index).difference(cats_df2_index))
    #main_list = list(set(list_2) - set(list_1))


    index_intersection = list(set(cats_df2_index) & set(scores_index))

    cats_df2 = cat_df.loc[index_intersection,:].sort_index()

    scores_series_vector_prod = scores_vector[index_intersection].sort_index()

    if len(outer) > 0:
        print(f'outer variable from sync_objects funct contains the following entries, which I should add to statusCombinationMatrix.ods {outer}')
    else:
        None

    return scores_series_vector_prod, cats_df2, outer

In [16]:
#function to convert "categories" df to matrix, then execute the multiplcation
#requires the vector and matrix to be "right-sized"
def apply_multiply(scores_vec:pd.Series, cat_df:pd.DataFrame):
    if scores_vec.shape[0] == cat_df.shape[0]:
        mat = cat_df.to_numpy()
#scores_series_vector = db resultset of status combinations and counts
        new_mat = np.matmul(np.transpose(scores_vec.array), mat)

        #returns a Series with the category balances
        return pd.Series(new_mat, index = cat_df.columns)
    
    else:
        raise TypeError("objects are not of appropriate size")

**Applying all the functions at once.** Ideally each iteration will return the measures of ea category, and preserves the measure date as either an index or field value

In [17]:
cats_df = import_treat_cat_df("statusCombinationMatrix_acctFlows.ods")
#scores_vector = package_scores_vector(df_dict['2023-03-31'])
#score_vec, cat_df = sync_objects(scores_vector,cats_df)
#scores_vector

### *Monthlies* dataframe: what each row shows 
1) the bottom-line balances  for the high-level accounts (active accounts, winbacks, leave, etc); and 
2) the change in the subcomponents (ex. new signups, new cancels, new suspensions, etc). 

In other words, the rows show the ending balances as well as what occurred in the month for the balances to change from previous month to 'current' month. A proposed QA is: I can take the negative net (multiply each of the values in the 'subcomponents' fields by -1) of the new cancellations, suspensions, signups, etc. add that to the activations for that period (remember activations are reported as of the final date of the given month) and tie to the **previous** month's ending activations balance

In [18]:
#iteratively apply the vector-matrix multiply to ea item in df_dict (first convert the stored df in df_dict to a score vector, then multiply by the categories matrix)
monthlies_dict = {}
#df_dict = freq df of email tallies by curr/prev actvitiy_calc
for k,v in df_dict.items():
    #package_scores_vector function that takes in a dataframe of a month's scores along with activity_calc fields, which then combines the TWO activity_calc fields into a tuple of the status combination. Extracts the unq.email field (int) and converts that into the scores Series with index corresponding to the combo tuple
    scores_vec = package_scores_vector(v)
    score_vec2, cats_df2,_  = sync_objects(scores_vec,cats_df)
    monthlies_dict[k]=apply_multiply(score_vec2, cats_df2)

In [19]:
monthlies_df = pd.DataFrame.from_dict(monthlies_dict,orient = 'index')

In [20]:
monthlies_df.to_csv('./monthlies_2024_test.csv')

### Inspect all the orphaned records and potentially add them to the master categories matrix

run the below operations, including to_csv exporting, then c+p the rows of the csv onto the 'statusCombinationMatrix_acctFlows.ods to consolidate all the status permutations

In [34]:
from itertools import chain

set(list(chain(*orphaned)))

#turn into a dataframe or vector, then c+p into the .ods file
pd.Series(list(chain(*orphaned))).drop_duplicates().to_csv('./orphaned.csv',index = False)